In [31]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2

## Conectar a AWS

In [32]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [33]:
aws_rds_conn = boto3.client('rds', aws_access_key_id = config.get('IAM','ACCESS_KEY'),
                            aws_secret_access_key = config.get('IAM','SECRET_KEY'),
                            region_name = 'us-east-2')

In [34]:
rds_intances_ids = []

aws_response = aws_rds_conn.describe_db_instances() # retorna un objeto [diccionario] iterable de las instancias

for response in aws_response['DBInstances']:
    rds_intances_ids.append(response['DBInstanceIdentifier'])

print(f'Instancias disponibles: {rds_intances_ids}')

Instancias disponibles: ['dbdim', 'dbtienda', 'sakila-db-pg-v']


In [35]:
try:
    response = aws_rds_conn.create_db_instance(
                                DBInstanceIdentifier=config.get('DIM','DB_INSTANCE_ID'),
                                DBName=config.get('DIM','DB_NAME'),
                                MasterUsername=config.get('DIM','DB_USERNAME'),
                                MasterUserPassword=config.get('DIM','DB_PASSWORD'),
                                Port=int(config.get('DIM','DB_PORT')),
                                DBInstanceClass='db.t3.micro',
                                Engine=config.get('DIM','DB_ENGINE'),
                                PubliclyAccessible=True,
                                AllocatedStorage=20,
                                VpcSecurityGroupIds=[config.get('VPC','SECURITY_GROUP')],
                            )
    print('Base de Datos creada exitosamente!!!')
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print('Error!!!',ex)

La instancia ya existe


### Access point dimensional

In [36]:
try:  
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('DIM','DB_INSTANCE_ID'))
    RDS_HOST_DBDIM = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOST_DBDIM)
except Exception as ex:
    print('Error!!!!',ex)

dbdim.cp6geq8ycm59.us-east-2.rds.amazonaws.com


#### Conexion a DB y creacion de tablas

In [37]:
import ddl_dimdb # py donde se encuentra el ddl

In [38]:
try:
    db_pg_conn = psycopg2.connect(
                            database=config.get('DIM','DB_NAME'),
                            user=config.get('DIM','DB_USERNAME'),
                            password=config.get('DIM','DB_PASSWORD'),
                            port=config.get('DIM','DB_PORT'),
                            host=RDS_HOSTNAME
                            )
    cursor = db_pg_conn.cursor()
    cursor.execute(ddl_dimdb.ddl)
    db_pg_conn.commit()
except Exception as ex:
    print('Error!!!',ex)